In [ ]:
import pyccl as ccl
import numpy as np
import matplotlib.pyplot as plt

This notebook generates the kernels which are to be provided to participants as part of the N5K challenge.

In [ ]:
#First set the fiducial cosmological parameters (use the same ones as in the SRD)
om_m=0.3156
w0=-1.0
h0=0.6727
A_s=2.12107*10**(-9) 
om_b=0.0492
n_s=0.9645

cosmo = ccl.Cosmology(Omega_c = om_m-om_b, Omega_b = om_b, n_s=n_s, A_s=A_s, h=h0, w0=w0)

In [ ]:
#Per-bin galaxy bias
b = [1.376695, 1.451179, 1.528404, 1.607983, 1.689579, 1.772899, 1.857700, 1.943754, 2.030887, 2.118943]

#Amplitude of NLA model (accounting for f_red already)
A_IA = 0.15

#pz parameters
zb_cl = 0.0
sigz_cl = 0.03
zb_sh = 0.0
sigz_sh = 0.05

In [ ]:
#Import redshift distrbutions
#These are the SRD Y10 redshift distrbutions binned as described in that document,
#convolved with the assumed fiducial photo-z model given there also.


dNdz_sh = np.loadtxt('./dNdz_srcs_LSSTSRD_zb0_sigz0.05.dat', unpack=True)
z_sh = dNdz_sh[0, :]
dNdz_sh = dNdz_sh[1:, :]
dNdz_cl = np.loadtxt('./dNdz_clust_LSSTSRD_zb0_sigz0.03.dat', unpack=True)
z_cl = dNdz_cl[0, :]
dNdz_cl = dNdz_cl[1:, :]

In [ ]:
# Plot these just to be sure they look okay:

plt.figure()
for n in dNdz_sh:
    plt.plot(z_sh, n)
plt.figure()
for n in dNdz_cl:
    plt.plot(z_sh, n)
plt.show()


In [ ]:
# Now that we have the z values for the clustering bins, set the galaxy bias vectors:

b_vecs = [b[i]*np.ones(len(z_cl)) for i in range(0,10)]

# And the IA amplitude:

A_IA_vecs = [A_IA*np.ones(len(z_sh)) for i in range(0,5)]

In [ ]:
# Now define the tracers

# Lenses
Cl_tracers = [ccl.NumberCountsTracer(cosmo,False, (z_cl, n),
                                     (z_cl, b_vecs[i]))
              for i, n in enumerate(dNdz_cl)]

# Sources
Sh_tracers = [ccl.WeakLensingTracer(cosmo, (z_sh, n), True,
                                    (z_sh, A_IA_vecs[i]))
              for i, n in enumerate(dNdz_sh)]

In [ ]:
# Extract the kernels:

# Input here is comoving radial distance:
chi_sh = ccl.comoving_radial_distance(cosmo, 1./(1.+z_sh))
chi_cl = ccl.comoving_radial_distance(cosmo, 1./(1.+z_cl))

kernels_cl = np.array([t.get_kernel(chi_cl)[0] for t in Cl_tracers])
kernels_sh = np.array([t.get_kernel(chi_sh)[0] for t in Sh_tracers])
kernels_ia = np.array([t.get_kernel(chi_sh)[1] for t in Sh_tracers])

In [ ]:
# Plot to make sure it looks okay:
plt.figure()
for k in kernels_cl:
    plt.plot(chi_cl, k)
plt.figure()
for k in kernels_sh:
    plt.plot(chi_sh, k)
plt.figure()
for k in kernels_ia:
    plt.plot(chi_sh, k)
plt.show()

In [ ]:
# Now save to files

kernels_cl_arr = np.column_stack((z_cl, chi_cl, kernels_cl.T))
kernels_sh_arr = np.column_stack((z_sh, chi_sh, kernels_sh.T))
kernels_ia_arr = np.column_stack((z_sh, chi_sh, kernels_ia.T))

header_cl='z,  chi (Mpc),  kernels 1-10'
header_src='z,  chi (Mpc),  kernels 1-5'

np.savetxt('./NumberCountKernels.dat',
           np.column_stack((z_cl, chi_cl, kernels_cl.T)),
           header = 'z,  chi (Mpc),  kernels 1-%d' % len(kernels_cl))
np.savetxt('./CosmicShearKernels.dat',
           np.column_stack((z_sh, chi_sh, kernels_sh.T)),
           header = 'z,  chi (Mpc),  kernels 1-%d' % len(kernels_sh))
np.savetxt('./IntrinsicAlignmentKernels.dat',
           np.column_stack((z_sh, chi_sh, kernels_ia.T)),
           header = 'z,  chi (Mpc),  kernels 1-%d' % len(kernels_ia))